In [14]:
library(sl3)
library(tmle3)
library(glue)
source("tstmle.R")

In [15]:
preprocess_data <- function(data){
    data$pm2_5 <- ifelse(data$pm2_5 >= 50, 1, 0)
    max_y <- max(data$breathing_rate)
    data$breathing_rate <- data$breathing_rate / max_y
    data$target <- data$breathing_rate
    return (data)
}

In [16]:
data <- preprocess_data(read.csv("data/experiments/experiment_one_DBCC14.csv"))

node_list <- list(
    W = c("activity_type"),
    A = "pm2_5",
    Y = "breathing_rate"
)
N <- nrow(data)
K <- 5

ate_spec <- tmle_ATE(treatment_level = 1, control_level=0)
folds <- origami::make_folds(fold_fun = folds_rolling_origin, n=N, first_window = N/2, validation_size = 5, gap = 5, batch = N/20)
learner_list = get_learner_list()

head(data)

,breathing_rate,activity_type,pm2_5,target
,<dbl>,<int>,<dbl>,<dbl>
1,0.5725079,0,0,0.5725079
2,0.7326675,0,0,0.7326675
3,0.6993243,0,0,0.6993243
4,0.5725079,0,0,0.5725079
5,0.5725079,1,0,0.5725079
6,0.5725079,1,0,0.5725079


In [17]:
#folds <- origami::make_folds(fold_fun = folds_rolling_origin, n=N, first_window = N/5, validation_size = 20, gap = 5, batch = N/25)
folds <- origami::make_folds(fold_fun = folds_rolling_window, n=N, window_size = N/5, validation_size=20, gap=10, batch=N/25)

length(folds)

[1] 20

Constructing initial estimate of $\bar g_0$, then initial estimate of $\bar Q_{L(K+1)}$

$$
L(\bar Q_{L(K+1)})(O^N) = - \sum_{t=1}^N\{Y(t)\log \bar Q_{L(K+1)}(O(t), C_o(t)) + (1-Y(t))\log (1-\bar Q_{L(K+1)})(O(t), C_o(t)) \}
$$

In [18]:
tmle_task <- make_tmle_task(ate_spec, data, node_list, folds, K)    

initial_likelihood <- ate_spec$make_initial_likelihood(
    tmle_task,
    learner_list
)

g <- initial_likelihood$get_likelihood(tmle_task, node = "A")

Given the initial estimate we define a parametric submodel. The universal least favourable submodel is given where $H$ is the clever covariate of the target parameter.

$$
\text{logit} \bar Q_{N,L(K+1), \epsilon} = \text{logit}\bar Q_{N,L(K+1)} + \epsilon H_{K+1}(\bar g_N)
$$

In [19]:
updater <- ate_spec$make_updater(cvtmle=FALSE)
targeted_likelihood <- ate_spec$make_targeted_likelihood(initial_likelihood, updater)
tmle_params <- ate_spec$make_params(tmle_task, targeted_likelihood)

updater$tmle_params <- tmle_params

tmle_fit <- fit_tmle3(tmle_task, targeted_likelihood, tmle_params, updater)
tmle_fit

A tmle3_Fit that took 1 step(s)
   type                param init_est    tmle_est         se       lower
1:  ATE ATE[Y_{A=1}-Y_{A=0}]        0 -0.01447533 0.01588325 -0.04560593
        upper psi_transformed lower_transformed upper_transformed
1: 0.01665528     -0.01447533       -0.04560593        0.01665528

The above can be thought of as a function 

$$
(C_o(t), O(t)) \rightarrow \bar Q^*_{N(L(K+1))}
$$

We define the following 

$$
\bar Q^*_{N,A(K)}(\cdot ) = \int_{a(K)} \bar Q ^*_{N,L(K+1)}(\cdot, a(K))\bar g^*_K(a(K) | C^*_{a,K}(t))
$$

This is effectively a weighted average of the probability treatment is given conditional on the context vector and counterfactual treatment $a(K)$.  

In [20]:
int_out_ak <- function(targeted_likelihood, g, k) {
    counterfactual_data <- preprocess_data(read.csv("data/experiments/experiment_one_DBCC14.csv"))

    folds <- folds_vfold(nrow(counterfactual_data), V=1)

    counterfactual_task <- make_tmle_task(ate_spec, counterfactual_data, node_list, folds, lag=k)
    counterfactual_vector <- matrix((targeted_likelihood$factor_list[["Y"]]$get_likelihood(counterfactual_task)))

    weights <- counterfactual_vector * matrix(g)
    return (weights)
    #weights <- weights / sum(weights) # Normalise
    #return(sum(weights * counterfactual_vector)) # Weighted Average 
}

In [21]:
q_nAk <- int_out_ak(targeted_likelihood, g, 5)

Which is the same as above but an estimate for treatment, and is now a only a function of
$C_o(t), L(t, 1: K)$ and $A(t, 1: K-1)$.

Given $\bar Q^*_{N,A(K)}$  we can define the appropriate loss for $\bar Q_L(K)$

$$
    L = - \text{sum}(y * \log(p) + (1-y) * \log(1-p)) \\  
    = - \sum_{t=1}^N\{\bar Q^*_{N,A(K)}\log \bar Q_{L(K)}+(1-Q^*_{N,A(K)})\log(1-\bar Q_{L(K+1)})\}(O(t),C_o(t)) 
$$

In [22]:
data$target <- q_nAk
node_list <- list(
    W = c("activity_type"),
    A = "pm2_5",
    Y = "target"
) 
tmle_fit_2 <- run_tstmle(ate_spec, data, node_list, markov_order= K-1)

<h2>Loop</h2>

Now we've done it once, lets loop it until we get to the final conditional mean.

In [23]:
K <- 5
data <- preprocess_data(read.csv("data/experiments/experiment_one_DBCC14.csv"))

ate_spec <- tmle_ATE(treatment_level = 1, control_level=0)
learner_list = get_learner_list()

node_list <- list(
    W = c("activity_type"),
    A = "pm2_5",
    Y = "target"
)

for (k in K:1){
    folds <- origami::make_folds(fold_fun = folds_rolling_origin, n=nrow(data), first_window = N, validation_size = 0, gap = 0, batch = 0)
    tmle_task <- make_tmle_task(ate_spec, data, node_list, folds, k)    

    initial_likelihood <- ate_spec$make_initial_likelihood(
        tmle_task,
        learner_list
    )

    g <- initial_likelihood$get_likelihood(tmle_task, node = "A")

    updater <- ate_spec$make_updater(cvtmle=FALSE)
    targeted_likelihood <- ate_spec$make_targeted_likelihood(initial_likelihood, updater)
    tmle_params <- ate_spec$make_params(tmle_task, targeted_likelihood)

    updater$tmle_params <- tmle_params

    tmle_fit <- fit_tmle3(tmle_task, targeted_likelihood, tmle_params, updater)
    print(k)
    print(tmle_fit)

    q_nAk <- int_out_ak(targeted_likelihood, g, k)
    data$target <- q_nAk
}